In [13]:
#Import system modules
import arcpy
from arcpy import env
import glob

In [14]:
#Needed wildcards and paths
origins = 'D:/ArcPy/arcpy_scripts/origins/testOrigins/*.shp'
destinations = 'D:/ArcPy/arcpy_scripts/destination/testDestination/*.shp'
layer_name = 'OD Cost Matrix'
output_csv = 'D:/ArcPy/arcpy_scripts/jenniferJennings/'

#Collecting files from directory
fListOrigins = glob.glob(origins)
fListDestinations = glob.glob(destinations)

In [15]:
env.overwriteOutput = True
arcpy.CheckOutExtension('Network')

u'CheckedOut'

In [16]:
fListOrigins[0][53:-4]

'Delaware'

In [17]:
#Solving OD Matrix for each state in a given year.
for index in range(len(fListOrigins)):
    
    #Creating OD Matrix
    print "Creating OD matrix for file %s"%index
    result_object  = arcpy.MakeODCostMatrixLayer_na(in_network_dataset="D:/ESRI_Data_2013/streetmap_na/data/streets",\
                               out_network_analysis_layer="OD Cost Matrix", \
                               impedance_attribute="Length", default_cutoff="25", \
                               default_number_destinations_to_find="", accumulate_attribute_name="Length", \
                               UTurn_policy="ALLOW_UTURNS",\
                               restriction_attribute_name="'Driving a Passenger Car';'Avoid Service Roads';'Avoid Pedestrian Zones';\
                               'Avoid Walkways';'Avoid Roads for Authorities';'Avoid Private Roads';'Avoid Unpaved Roads';\
                               'Through Traffic Prohibited';'Avoid Express Lanes';'Avoid Carpool Roads'", 
                               hierarchy="USE_HIERARCHY", hierarchy_settings="", \
                               output_path_shape="NO_LINES", time_of_day="")
    
    layer_object = result_object.getOutput(0)
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]
    linesLayerName = sublayer_names['ODLines']
    
    #Adding Origin and Destination to the OD Matrix
    print "Adding Origin for file %s"%fListOrigins[index]
    arcpy.na.AddLocations(layer_object,  origins_layer_name, fListOrigins[index], 
                          "Name ncessch #;TargetDestinationCount # #", "1000 Meters")
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,  origins_layer_name)
    
    print "Adding Destinations for file %s"%fListDestinations[index]
    arcpy.AddLocations_na(layer_object, destinations_layer_name, fListDestinations[index],
                      field_mappings="Name ncessch #", search_tolerance="1000 Meters")
    
    #Solving OD Matrix to obtain Distance metric
    print "Starting Network Analysis for file %s"%str(fListOrigins[index][53:-4])
    arcpy.na.Solve(layer_object)
    print "Completed Network Analysis for file %s"%str(fListOrigins[index][53:-4])
    
    #Saving distance metric to csv
    output_csv = 'D:/ArcPy/arcpy_scripts/jenniferJennings/outputCSV/'+str(fListOrigins[index][53:-4])+'_14.csv'
    fields = ["Name", "Total_Length"]
    headers = ['Origin', 'Destination', 'Length']
    for lyr in arcpy.mapping.ListLayers(layer_object):
        if lyr.name == linesLayerName:
            with open(output_csv, 'w') as f:
                f.write(','.join(headers)+'\n') # csv headers
                with arcpy.da.SearchCursor(lyr, fields) as cursor:
                    print "Successfully created lines searchCursor.\nExporting to " + output_csv 
                    for i, row in enumerate(cursor):
                        string = row[0].split('-')
                        string.append(row[1])
                        f.write(','.join([str(r) for r in string])+'\n')
    print "Saved file %s"%(output_csv)

Creating OD matrix for file 0
Adding Origin for file D:/ArcPy/arcpy_scripts/origins/testOrigins\origin_14_Delaware.shp
Adding Destinations for file D:/ArcPy/arcpy_scripts/destination/testDestination\destination_14_Delaware.shp
Starting Network Analysis for file Delaware
Completed Network Analysis for file Delaware
Successfully created lines searchCursor.
Exporting to D:/ArcPy/arcpy_scripts/jenniferJennings/outputCSV/Delaware_14.csv
Saved file D:/ArcPy/arcpy_scripts/jenniferJennings/outputCSV/Delaware_14.csv
Creating OD matrix for file 1
Adding Origin for file D:/ArcPy/arcpy_scripts/origins/testOrigins\origin_14_District of Columbia.shp
Adding Destinations for file D:/ArcPy/arcpy_scripts/destination/testDestination\destination_14_District of Columbia.shp
Starting Network Analysis for file District of Columbia
Completed Network Analysis for file District of Columbia
Successfully created lines searchCursor.
Exporting to D:/ArcPy/arcpy_scripts/jenniferJennings/outputCSV/District of Columbi